In [35]:
import pysam
import numpy as np
from collections import Counter
vant = np.loadtxt("/Users/SemiQuant/Bioinformatics/Projects/Hasan/snps.tsv", delimiter="\t", 
          dtype={'names': ('CHROM', 'POS', 'REF', 'ALT', 'TYPE'),
          'formats': ('<U100', 'int', 'U10', 'U10', 'U10')}, skiprows=1)

# input file is the reference name, reference based positions and mutations and type

samfile = pysam.AlignmentFile("/Users/SemiQuant/Downloads/test/test.bam", "rb")

# TODO
# add in procedures for indels
# add in better output format, make it a matrix
# add in reference sequence so can determine more info later


phase_out = []
for chrom in set(vant["CHROM"]):
    seq_out = []
    pos = vant["POS"][vant["CHROM"] == chrom]
    call = vant["ALT"][vant["CHROM"] == chrom]
    ref = vant["REF"][vant["CHROM"] == chrom]
    try:
        for read in samfile.fetch(chrom, until_eof=True):
            try:
                #convert to positions in the read
                pos_conv = read.query_alignment_start + read.get_reference_positions()[0] #unaligned leading bases + position of first alignment on reference
                pos_conv = pos - pos_conv
                phase = ""
                call_c = 0
                for p in pos_conv:
                    try:
                        # we dont want any bases that were filtered out so only take those that match the input
                        if read.query_sequence[p] in call[call_c].split(',') or read.query_sequence[p] == ref[call_c]: # in case more than on allele
                            phase += read.query_sequence[p]
                        else:
                            phase += "N"
                    except Exception:
                        phase += "N"
                    call_c += 1
                seq_out.append(phase)
            except Exception:
                continue
        # phase_out.append([[chrom], [Counter(seq_out)]])
        phase_out.append([[chrom], list(Counter(seq_out).items())])
    except Exception:
        print(chrom, " not found in bam file!")

rpoC  not found in bam file!


In [ ]:
phase_out

In [115]:
import pysam
import numpy as np
from collections import Counter
samfile = pysam.AlignmentFile("/Users/SemiQuant/Downloads/BDQ_haplo/TB-B240-054_S26-bowtie2.bam", "rb")


In [117]:
count=0
for read in samfile.fetch(until_eof=True):
    count+=1
    if read.qname == 'M02061:126:000000000-BVK9B:1:1102:15579:11359':
        for cigar_operations in read.cigar:
            (type_cigar_op,length_cigar_op)=cigar_operations
            print(cigar_operations)
            print(type_cigar_op)

#             print(read.query_sequence[p])
#                 if type_cigar_op==2 :# DELETION
#         for p in range(0, 240):
#             print(read.query_alignment_start)
#             print(p)
#             print(read.query_sequence[p])
                    
#     if count >10:
#         break

In [140]:
for pileupcolumn in samfile.pileup('Rv0678_amplicon4', 111, 112):
    for pileupread in pileupcolumn.pileups:
        #so this is each base in the read in the pileup
        if pileupread.alignment.query_name == 'M02061:126:000000000-BVK9B:1:1102:15579:11359':
#             print(pileupread)
#             print(pileupread.query_position)
#             print ('\tbase in read %s = %s' %
#                   (pileupread.alignment.query_name,
#                    pileupread.alignment.query_sequence[pileupread.query_position]))

SyntaxError: unexpected EOF while parsing (<ipython-input-140-83bc4e11ba6b>, line 9)

False

In [146]:
import pysam
from collections import Counter
import pandas as pd
samfile = pysam.AlignmentFile("/Users/SemiQuant/Downloads/BDQ_haplo/TB-B240-054_S26-bowtie2.bam", "rb")

# var_dic will be snps/indels, if also adding reference then edit the below
# import numpy as np
# vant = np.loadtxt("/Users/SemiQuant/Bioinformatics/Projects/Hasan/snps.tsv", delimiter="\t", 
#           dtype={'names': ('CHROM', 'POS', 'REF', 'ALT', 'TYPE'),
#           'formats': ('<U100', 'int', 'U10', 'U10', 'U10')}, skiprows=1)
var_dic = {}
with open("/Users/SemiQuant/Bioinformatics/Projects/Hasan/snps.tsv") as f:
    next(f) # skip header line
    for line in f:
        (CHROM, POS, REF, ALT, TYPE) = line.split()
        try:
            var_dic[CHROM][int(POS)].setdefault('call', []).append(ALT)
        except:
            try:
                var_dic[CHROM][int(POS)] = {'call' : [ALT]}
            except:
                var_dic[CHROM] = {int(POS) : {'call' : [ALT]}}


In [154]:
var_dic

In [148]:
for chrom in var_dic:
    if chrom not in samfile.references:
        print(chrom, " not found in bam file!")
    else:
        for i in var_dic[chrom]:
            for pileupcolumn in samfile.pileup(chrom, i, i+1):
                for pileupread in pileupcolumn.pileups:
                    if pileupread.query_position == i:
                        if not pileupread.is_del: # if reference then add
                            read_call_tmp = pileupread.alignment.query_sequence[pileupread.query_position]
                            try:
                                #  check if it matches the call in there as pairs have same name
                                if read_call_tmp == var_dic[chrom][i][pileupread.alignment.query_name]:
                                    break
                                else:
                                    var_dic[chrom][i][pileupread.alignment.query_name] = "N"
                            except:
                                if pileupread.is_refskip == 0: # if reference then add, not sure if this is working
                                    var_dic[chrom][i][pileupread.alignment.query_name] = read_call_tmp
                                elif read_call_tmp in var_dic[chrom][i]["call"]:
                                    var_dic[chrom][i][pileupread.alignment.query_name] = read_call_tmp
                                else:
                                    var_dic[chrom][i][pileupread.alignment.query_name] = "N"
                        else:
                            try:
                                #  check if it matches the call in there as pairs have same name
                                if read_call_tmp == var_dic[chrom][i][pileupread.alignment.query_name]:
                                    break
                                else:
                                    var_dic[chrom][i][pileupread.alignment.query_name] = "N"
                            except:
                                if read_call_tmp == var_dic[chrom][i][pileupread.alignment.query_name]:
                                    break
                                else:
                                    var_dic[chrom][i][pileupread.alignment.query_name] = "-"


                                    

rpoC  not found in bam file!


111
112
114
116


In [151]:
var_df = pd.DataFrame.from_dict({(i,j): var_dic[i][j] 
                           for i in var_dic.keys() 
                           for j in var_dic[i].keys()},
                       orient='columns')

In [ ]:
var_df

In [ ]:
var_df[1:]

In [ ]:
import pysam
import numpy as np
from collections import Counter
vant = np.loadtxt("/Users/SemiQuant/Bioinformatics/Projects/Hasan/snps.tsv", delimiter="\t", 
          dtype={'names': ('CHROM', 'POS', 'REF', 'ALT', 'TYPE'),
          'formats': ('<U100', 'int', 'U10', 'U10', 'U10')}, skiprows=1)


In [ ]:
vant

In [ ]:
var_dic

In [32]:
chrom="rpoB"

In [35]:
var_dic[chrom][i]

['G']